1、张量
2、自动微分
3、nn包
##### 步骤
不使用自动微分的情况下
1.输入数据  --》(使用模型包装)
2.设置权重并初始化
3.前向传递得到输出
4.计算loss --》定义loss函数
5.反向传播计算梯度--》自动微分
6.更新参数--》定义优化器

使用自动微分：
1.输入数据
2.设置权重并初始化，标记requires_grad=True
3.前向传递得到输出
4.计算loss
5.loss.backfward()
6.更新参数

使用优化器情况下
1.输入数据
2.定义model：
3.定义优化器
4.计算loss函数
5.训练：输入数据，使用loss函数，反向传播，使用优化器更新参数


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as f
import numpy as np

N,D_in,H,D_out=64,1000,100,10

In [2]:
#不使用GPU,

x=np.random.randn(N,D_in)
y=np.random.randn(N,D_out)

w1=np.random.randn(D_in,H)
w2=np.random.randn(H,D_out)

learning_rate=1e-6
for t in range(500):
    # 前向传递
    h=x.dot(w1)
    h_relu=np.maximum(h,0)
    y_pred=h_relu.dot(w2)
    #计算loss
    loss=np.square(y_pred-y).sum()
    print(t,loss)
    #计算梯度
    grad_y_pred=2.0*(y_pred-y)

    grad_w2=h_relu.T.dot(grad_y_pred)
    grad_h_relu=grad_y_pred.dot(w2.T)
    grad_h=grad_h_relu.copy()
    grad_h[h<0]=0
    grad_w1=x.T.dot(grad_h)
    #更新参数
    w1-=learning_rate*grad_w1
    w2-=learning_rate*grad_w2
    break

0 27878547.280449398


In [11]:
#自动微分
device=torch.device('cuda:0')
x=torch.randn(N,D_in,device=device,dtype=torch.float)
y=torch.randn(N,D_out,device=device,dtype=torch.float)
#添加 requires_grad=True
w1=torch.randn(D_in,H,device=device,dtype=torch.float,requires_grad=True)
w2=torch.randn(H,D_out,device=device,dtype=torch.float,requires_grad=True)
learning_rate=1e-6
for t in range(500):
    # 前向传递
    h=x.mm(w1)
    h_relu=h.clamp(min=0)
    y_pred=h_relu.mm(w2)

    #计算loss
    loss=(y_pred-y).pow_(2).sum()
    print(t,loss.item())
    #自动计算梯度
    loss.backward()
    
    with torch.no_grad():
        #参数更新
        w1-=learning_rate*w1.grad
        w2-=learning_rate*w2.grad
        #梯度清零
        w1.grad.zero_()
        w2.grad.zero_()
    break

0 29751810.0


In [13]:
#最抽象版本
device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
x=torch.randn(N,D_in,device=device,dtype=torch.float)
y=torch.randn(N,D_out,device=device,dtype=torch.float)
learning_rate=1e-6
#定义模型
model=torch.nn.Sequential(
    torch.nn.Linear(D_in,H),
    torch.nn.ReLU(),
    torch.nn.Linear(H,D_out),)
model.to(device)
#定义loss函数
loss_fn=torch.nn.MSELoss(reduction='sum')
#定义优化器
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)

for t in range(500):
    y_pred=model(x)
    loss=loss_fn(y,y_pred)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    break

0 636.65478515625
